Question Answering bot

In [1]:
 # Question-Answering

In [2]:
!pip install langchain
!pip install langchain_community
!pip install -U langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49

In [3]:
!pip install transformers faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 21.3 MB/s eta 0:00:00


In [4]:
#!pip freeze > requirement.txt

In [4]:
#Loading Data

from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("http://jalammar.github.io/illustrated-transformer/")
data = loader.load()

In [5]:
# splitting Data and creating vector store

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(data)

In [6]:
# Storing

# Import FAISS from Langchain Vectorstore
from langchain.vectorstores import FAISS

from langchain_huggingface import HuggingFaceEmbeddings
from tqdm import tqdm, trange
from transformers import AutoTokenizer

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

tokenizer = AutoTokenizer.from_pretrained(model_name, clean_up_tokenization_spaces=True)

hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


 # Creating a vector store
vectorstore = FAISS.from_documents(documents=all_splits, embedding=hf) ## hf are the hugging face embeddings

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# REtrieval

question = "What are transformers?"
docs = vectorstore.similarity_search(question)
docs



[Document(metadata={'source': 'http://jalammar.github.io/illustrated-transformer/', 'title': 'The Illustrated Transformer – Jay Alammar – Visualizing machine learning one concept at a time.', 'description': 'Discussions:\nHacker News (65 points, 4 comments), Reddit r/MachineLearning (29 points, 3 comments)\n\n\nTranslations: Arabic, Chinese (Simplified) 1, Chinese (Simplified) 2, French 1, French 2, Italian, Japanese, Korean, Persian, Russian, Spanish 1, Spanish 2, Vietnamese\n\nWatch: MIT’s Deep Learning State of the Art lecture referencing this post\n\nFeatured in courses at Stanford, Harvard, MIT, Princeton, CMU and others\n\nIn the previous post, we looked at Attention – a ubiquitous method in modern deep learning models. Attention is a concept that helped improve the performance of neural machine translation applications. In this post, we will look at The Transformer – a model that uses attention to boost the speed with which these models can be trained. The Transformer outperform

In [8]:
# Generation Using Roberta language model


# For downloading the models from HF Hub
!pip install huggingface_hub==0.23.5 -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 kB 26.0 MB/s eta 0:00:00


In [9]:
# Set the environment variable
from google.colab import userdata
import os

# Ensure the token is correctly retrieved and assigned to the environment variable
token = userdata.get('HF_TOKEN')
if token:
  os.environ['HUGGINGFACEHUB_API_TOKEN'] = token
else:
  print("HuggingFace token not found in userdata.")

In [10]:
# prompt: use distilbert-base-uncased-distilled-squad: from HuggingFaceHub for question answering system

from transformers import pipeline

# Replace with your desired model from HuggingFace Hub
model_name = "distilbert-base-uncased-distilled-squad"

# Initialize the question-answering pipeline
qa_pipeline = pipeline("question-answering", model=model_name)

# Perform question-answering using the pipeline
def answer_question(question, context):
  result = qa_pipeline({
      'question': question,
      'context': context
  })
  return result['answer']


# Example usage
question = "What are transformers?"
context = docs[0].page_content
answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer}")


config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Question: What are transformers?
Answer: Transformer


In [11]:
# Summary



from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationSummaryMemory

# Assuming qa_pipeline is the question answering pipeline object from previous code
memory = ConversationSummaryMemory(
    llm=HuggingFacePipeline(pipeline=qa_pipeline), # Pass the pipeline object to HuggingFacePipeline
    memory_key="chat_history",
    return_messages=True
)

<ipython-input-11-aae263d10e44>:10: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm=HuggingFacePipeline(pipeline=qa_pipeline), # Pass the pipeline object to HuggingFacePipeline
<ipython-input-11-aae263d10e44>:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(


In [12]:
#Conversational Retrieval Chain
# chat history + new context

from langchain.chains import ConversationalRetrievalChain
# Define the llm variable using the HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=qa_pipeline)

retriever = vectorstore.as_retriever()
chat = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory,
    verbose=True
)

In [24]:
type(docs)

4

## Same question

In [29]:
question = "What are transformers?"
context = docs[0].page_content
answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer}")

Question: What are transformers?
Answer: Transformer


In [25]:
question = "What are transformers?"
context = docs[1].page_content
answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer}")

Question: What are transformers?
Answer: Narrated Transformer


In [26]:
question = "What are transformers?"
context = docs[2].page_content
answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer}")

Question: What are transformers?
Answer: TensorFlow


In [27]:
question = "What are transformers?"
context = docs[3].page_content
answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer}")

Question: What are transformers?
Answer: Hacker News


## Search in whole document

In [30]:
question = "What are transformers?"
context = ""
for doc in docs:
  context += doc.page_content

answer = answer_question(question, context)

print(f"Question: {question}")
print(f"Answer: {answer}")


Question: What are transformers?
Answer: Narrated Transformer
